In [1]:

from xgboost import XGBClassifier
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
import pandas as pd
import numpy as np
import sklearn
import utils
from sklearn import linear_model
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn import tree
from sklearn import linear_model
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn import tree
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve,roc_auc_score,precision_score,recall_score,auc
from matplotlib import pyplot
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE

In [2]:
dataSet = pd.read_csv('data/train_all_features.csv')
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26884 entries, 0 to 26883
Data columns (total 83 columns):
CUST_COD                     26884 non-null float64
LIMIT_BAL                    26884 non-null float64
SEX                          26884 non-null float64
PAY_DEC                      26884 non-null float64
PAY_NOV                      26884 non-null float64
PAY_OCT                      26884 non-null float64
PAY_SEP                      26884 non-null float64
PAY_AUG                      26884 non-null float64
PAY_JUL                      26884 non-null float64
BILL_AMT_DEC                 26884 non-null float64
BILL_AMT_NOV                 26884 non-null float64
BILL_AMT_OCT                 26884 non-null float64
BILL_AMT_SEP                 26884 non-null float64
BILL_AMT_AUG                 26884 non-null float64
BILL_AMT_JUL                 26884 non-null float64
PAY_AMT_DEC                  26884 non-null float64
PAY_AMT_NOV                  26884 non-null float64
PAY_AMT

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataSet, test_size = 0.2)
target = 'DEFAULT PAYMENT JAN'
features = test.columns[test.columns!=target]
train.head()

,CUST_COD,LIMIT_BAL,SEX,PAY_DEC,PAY_NOV,PAY_OCT,PAY_SEP,PAY_AUG,PAY_JUL,BILL_AMT_DEC,...,BILL_AMT_SIGMA,PAY_AMT_AR,PAY_AMT_I,PAY_AMT_MA,PAY_AMT_SIGMA,PAY_AR,PAY_I,PAY_MA,PAY_SIGMA,TOTAL_AMT_PAID
16143,37558967.0,7155.264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6700.189210,...,927.520160,0.0,2.0,0.0,20.680175,0.0,0.0,0.0,0.000000,1210.134024
26206,11963277.0,5068.312,1.0,1.0,-2.0,-1.0,-1.0,-2.0,-2.0,0.000000,...,6.110980,0.0,0.0,0.0,6.110980,0.0,0.0,0.0,1.067187,16.397480
20755,42811530.0,8645.944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5860.340098,...,341.327090,0.0,0.0,0.0,41.670080,0.0,0.0,0.0,0.000000,1050.899586
8686,50996822.0,1490.680,0.0,4.0,3.0,2.0,0.0,0.0,0.0,1503.171898,...,20.558704,0.0,0.0,0.0,24.342702,0.0,1.0,0.0,0.836661,178.881600
25603,26151434.0,1490.680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,905.111082,...,174.686661,0.0,0.0,0.0,17.355738,0.0,0.0,0.0,0.000000,363.725920


In [4]:
xTr = train[features]
yTr = train[target]
xTe = test[features]
yTe = test[target]

In [ ]:
from monster import Kraken

RANDOM_SEED = 2323

params_xgb = {
	'base_score' : [0.3,0.2,0.4],
	'max_depth' : [3,2],
	'n_estimators' : [80,85,90,95],
	'seed' : [RANDOM_SEED],
    'n_jobs': [-1],
}




models = [XGBClassifier]
params = [params_xgb]

krk = Kraken(models, params, random_state = RANDOM_SEED)
krk.fit(xTr, yTr)

Converting X in numpy array
Converting y in numpy array
Created test holdout. % positive class in test holdout = 0.22198977219897723


========== Starting grid search for <class 'xgboost.sklearn.XGBClassifier'> ==========
Evaluating model with parameters: {'base_score': 0.3, 'max_depth': 3, 'n_estimators': 80, 'n_jobs': -1, 'seed': 2323}
Training fold #1	0.524547803617571
Training fold #2	0.55980271270037
Training fold #3	0.5501770956316411
Training fold #4	0.5862944162436549
Training fold #5	0.539240506329114
Training fold #6	0.5583634175691936
Training fold #7	0.543640897755611
Training fold #8	0.5499383477188655
Training fold #9	0.5489199491740787
Training fold #10	0.5545023696682464
Average score: 0.5515427516408347     Average threshold: 0.27482643723487854
Evaluating model with parameters: {'base_score': 0.3, 'max_depth': 3, 'n_estimators': 85, 'n_jobs': -1, 'seed': 2323}
Training fold #1	0.53125
Training fold #2	0.5590062111801243
Training fold #3	0.5380249716231555
Training fol

# XGBoost

#      "text/plain": [
       "XGBClassifier(base_score=0.3, booster='gbtree', colsample_bylevel=1,\n",
       "       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,\n",
       "       max_depth=50, min_child_weight=1, missing=None, n_estimators=200,\n",
       "       n_jobs=4, nthread=4, objective='binary:logistic', random_state=0,\n",
       "       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,\n",
       "       subsample=1)"


In [ ]:
params = {
}

grid = utils.ThresholdGridCV(XGBClassifier, params)
grid.fit(xTr, yTr)
grid._scores

# grid._threshold

In [ ]:
from sklearn.metrics import f1_score
yPre=xg.predict(xTe)
f1_score(yTe, yPre)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(xTr, yTr)
importances = model.feature_importances_
indices = np.argsort(importances)[-40:]
tra = train.ix[:,indices]
xTr = train[features]
yTr = train[target]
tra.info()


In [ ]:
indices = np.argsort(importances)[-20:]
tra = train.ix[:,indices]
xTr = train[features]
yTr = train[target]
train.info()

# Confusion matrix Accuracy f1

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
ada2 = AdaBoostClassifier(n_estimators=500, random_state=0)
ada2 = ada2.fit(xTr,yTr)
yPre=ada2.predict(xTe)
conf=confusion_matrix(yTe, yPre)

In [ ]:
from sklearn.metrics import f1_score
f1_score(yTe, yPre)

In [ ]:
import utils
utils.get_best_threshold(ada2,xTe,yTe)
ada2.getparam()

# Evaluation using crossvalidation

In [ ]:
dt_entropy = tree.DecisionTreeClassifier()

In [ ]:
dt_entropy = tree.DecisionTreeClassifier('entropy',max_depth=10)
dt_entropy = dt_entropy.fit(x,y)
xval_entropy = model_selection.cross_val_score(dt_entropy, x, y, cv=KFold(n_splits=10, shuffle=True, random_state=1234))
print("Decision Tree (Entropy) Accuracy = %3.2f +/- %.4f" %(np.average(xval_entropy),np.std(xval_entropy)))

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
rf = rf.fit(x,y)
rf_eval = model_selection.cross_val_score(rf, x, y, cv=StratifiedKFold(n_splits=10,random_state=52725,shuffle=True))
print("Random Forest\t%4.3f\t%4.3f" % (np.average(rf_eval), np.std(rf_eval)))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=100, random_state=0)
ada = ada.fit(x,y)
ada_eval = model_selection.cross_val_score(ada, x, y, cv=StratifiedKFold(n_splits=10,random_state=52725,shuffle=True))
print("Ada\t%4.3f\t%4.3f" % (np.average(rf_eval), np.std(rf_eval)))

## 